In [ ]:
# Run once with real value then revert.
KEY = 'redacted'


In [2]:
# Common imports, constants and functionality.

import datetime
import itertools
import json
import requests

BASE = 'https://api.torn.com/v2'

class Missing:
    def _propagate(self):
        return self

    def __getattr__(self, name):
        return self._propagate
    
    def __bool__(self):
        return False


class Jsh:
    @classmethod
    def wrap(cls, inner):
        return Jsh(inner) if Jsh.wrappable(inner) else inner
    
    @classmethod
    def wrappable(cls, inner):
        return inner.__class__.__name__ == 'dict'
    
    @classmethod
    def fetch(cls, url):
        res = requests.get(url)
        if not res.ok:
            raise(f'Failed to fetch "{url}": {res.error}')
        data = json.loads(res.content)
        if 'error' in data:
            raise(f'Error returned from "{url}": {data["error"]}')
        return Jsh(data)
    
    def __init__(self, inner):
        assert(Jsh.wrappable(inner))
        self._inner = inner

    def __getattr__(self, name):
        if name == '_inner':
            return self._inner
        if not name in self._inner:
            return Missing()
        v = self._inner[name]
        match v.__class__.__name__:
            case 'dict': return Jsh(v)
            case 'list': return [Jsh.wrap(item) for item in v]
            case _: return v
    
    def __repr__(self):
        return str(self._inner)
    
    def has(self, name):
        return name in self._inner
    
    def dumps(self):
        return json.dumps(self._inner, indent=2)


In [3]:
url = f'{BASE}/racing/tracks?key={KEY}'
tracks = {}

try:
    jsh = Jsh.fetch(url)
except Exception as e:
    print(f'Failed to fetch "{url}": {e}')
    exit(1)
if not jsh.has('tracks'):
    print(f'No "tracks" in:\n{jsh.dumps()}')
    exit(1)
tracks.update([(track.id, track.title) for track in jsh.tracks])

print(f'Found {len(tracks)} tracks(s):')
for (id, name) in tracks.items():
    print(f'{name} [{id}]')


Found 16 tracks(s):
Uptown [6]
Withdrawal [7]
Underdog [8]
Parkland [9]
Docks [10]
Commerce [11]
Two Islands [12]
Industrial [15]
Vector [16]
Mudpit [17]
Hammerhead [18]
Sewage [19]
Meltdown [20]
Speedway [21]
Stone Park [23]
Convict [24]


In [4]:
url = f'{BASE}/faction/members?key={KEY}'
members = {}

try:
    jsh = Jsh.fetch(url)
except Exception as e:
    print(f'Failed to fetch "{url}": {e}')
    exit(1)
if not jsh.has('members'):
    print(f'No "members" in:\n{jsh.dumps()}')
    exit(1)
members.update([(member.id, member.name) for member in jsh.members])

print(f'Found {len(members)} members(s)')

for (id, name) in members.items():
    print(f'{name} [{id}]')


Found 33 members(s)
Big_Tex1429 [1513734]
EddieLee706 [1591144]
Mingle [2122084]
Talendrife [2143613]
Lexii [2637832]
Pvpkiller [2682292]
DukeSilver [2733982]
Sofro [2734960]
Socharis [2801249]
AUSTINEZ [2843077]
Rhysand [2919031]
Kitana1988 [3037508]
Sniffles666 [3249261]
Giftedbasicbee [3360473]
Shodgson [3509557]
Tsik [3527061]
HUMBLED-sama [3542978]
VeronicaMars [3562079]
Fikowi [3602025]
DSoul [3611176]
Schwartzenadder [3623941]
ShaboingBong [3646316]
Cringe_Lord [3689754]
SweetPetite [3713328]
rsty [3743574]
NTHRITE [3799007]
Boduki [3825722]
000Machie000 [3829798]
djoul_t60 [3842922]
Djiinni [3871156]
Parzival175 [3873916]
Ahmad1Albustami [3894597]
Summoner02 [3898384]


In [ ]:
url = f'{BASE}/user/races?limit=100'
cutoff = datetime.datetime(2025, 11, 8, 0, 0, 0).timestamp()
remaining = 16
done = False
rids = []

while url and not done:
    url += f'&key={KEY}'
    try:
        jsh = Jsh.fetch(url)
    except Exception as e:
        print(f'Failed to fetch "{url}": {e}')
        exit(1)
    if not jsh.has('races'):
        print(f'No "races" in:\n{jsh.dumps()}')
        exit(1)
    for race in jsh.races:
        if race.schedule.start < cutoff or not remaining:
          done = True
          break
        if race.title.startswith('Human Factor RL #'):
            print(f'{race.id}: {race.title} @ {tracks[race.track_id]} ({race.status})')
            rids.append(race.id)
            remaining -= 1
    url = jsh._metadata.links.prev

rids = list(reversed(rids))
print(rids)


16894378: Human Factor RL #1 @ Two Islands (finished)
[16894378]


In [13]:
n = len(rids)
hdrs1 = list(itertools.repeat("", 2*n+1))
hdrs2 = ["name"]
for i in range(0, n):
    hdrs2.extend(["pos", "time"])
results = dict(
    (id, list(itertools.chain([name], itertools.repeat("", 2*len(rids)+1))))
    for (id, name) in members.items()
)
#for id, name in members.items():
#    results[id][0] = name
for i, rid in enumerate(rids):
    url = f'{BASE}/racing/{rid}/race?key={KEY}'
    try:
        jsh = Jsh.fetch(url)
    except Exception as e:
        print(f'Failed to fetch "{url}": {e}')
        exit(1)
    if not jsh.has('race'):
        print(f'No "race" in:\n{jsh.dumps()}')
        exit(1)
    hdrs1[2*i+1] = tracks[jsh.race.track_id]
    for res in jsh.race.results:
        if res.driver_id in members:
            results[res.driver_id][2*i+1:2*i+3] = [
                str(res.position or ""),
                "DNF" if res.has_crashed else str(res.race_time or ""),
            ]

print(','.join(hdrs1))
print(','.join(hdrs2))
for res in results.values():
    print(','.join(res))


,Two Islands,
name,pos,time
Big_Tex1429,9,12201.91,
EddieLee706,,,
Mingle,2,10975.75,
Talendrife,,,
Lexii,3,11070.75,
Pvpkiller,13,25738.68,
DukeSilver,6,11683.71,
Sofro,14,26635.15,
Socharis,1,10961.4,
AUSTINEZ,,,
Rhysand,,,
Kitana1988,,,
Sniffles666,,,
Giftedbasicbee,,,
Shodgson,,,
Tsik,,,
HUMBLED-sama,,,
VeronicaMars,,,
Fikowi,12,14208.8,
DSoul,,,
Schwartzenadder,5,11500.62,
ShaboingBong,,,
Cringe_Lord,,,
SweetPetite,4,11330.57,
rsty,,,
NTHRITE,,,
Boduki,10,12417.97,
000Machie000,11,13312.36,
djoul_t60,,,
Djiinni,7,11829.58,
Parzival175,,,
Ahmad1Albustami,8,11963.35,
Summoner02,,,


In [7]:
url = f'{BASE}/faction/chains?limit=100&sort=DESC'
cutoff = datetime.datetime(2025, 10, 24, 0, 0, 0)
chains = []

while url:
    url += f'&key={KEY}'
    try:
        jsh = Jsh.fetch(url)
    except Exception as e:
        print(f'Failed to fetch "{url}": {e}')
        exit(1)
    if not jsh.has('chains'):
        print(f'No "chains" in:\n{jsh.dumps()}')
        exit(1)
    chains += jsh.chains
    url = jsh._metadata.links.prev

print(f'Found {len(chains)} chain(s) in total')

(c, s, r) = itertools.repeat(0, 3)
for chain in chains:
    if chain.start < cutoff.timestamp():
        break
    c += 1
    s += chain.chain
    r += chain.respect

print(f'{c} chain(s) totaling {s} hit(s) and {r} respect found after {cutoff.strftime("%c")}')


Found 882 chain(s) in total
149 chain(s) totaling 13645 hit(s) and 34567.81 respect found after Fri Oct 24 00:00:00 2025
